In [ ]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'
targets = get_column_values(path, 'descriptions_measures', 'B')
encoding_method = 'word2vec'
version = 'word2vec-google-news-300'




similarities_dict = get_similarity_scores(text=Q1,documents=targets,encoding_method=encoding_method,version=version)

top_10 = get_first_n_keys(similarities_dict, 10)


top_10

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

processed_text = preprocess_text(Q1)



w2v = api.load("word2vec-google-news-300")
vocab = w2v.index_to_key
words = Q1.split()


count = sum(1 for word in processed_text if word in w2v)
existing_words = [word for word in processed_text if word in w2v]
difference = len(words) - count

pr, orig = get_unique_items(words, processed_text)

pr

In [5]:
from helper import *

path='../reports/corpus.xlsx'

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

Q2 = 'Provide a report that displays the number of accounts not in running settlements for the Earth portfolio, \
  their expected monthly payments at the end of the examined month and their actual monthly payments per DCA, for September 2020.'

ground_truth = {'R1':[preprocess_text(element) for element in get_column_values(path, 'R1', 'C')],
                        'R2':[preprocess_text(element) for element in get_column_values(path, 'R2', 'C')],
                        'R3':[preprocess_text(element) for element in get_column_values(path, 'R3', 'C')],
                        'R4':[preprocess_text(element) for element in get_column_values(path, 'R4', 'C')],
                        'R5':[preprocess_text(element) for element in get_column_values(path, 'R5', 'C')],
                        'R6':[preprocess_text(element) for element in get_column_values(path, 'R6', 'C')],
                        'R7':[preprocess_text(element) for element in get_column_values(path, 'R7', 'C')],
                        'R8':[preprocess_text(element) for element in get_column_values(path, 'R8', 'C')],
                        'R9':[preprocess_text(element) for element in get_column_values(path, 'R9', 'C')]}   

processed_text = preprocess_text(Q2)

ground_truth['R4']


['settlement duration 6 month included',
 'settlement duration 7 12 month included',
 'settlement duration 13 36 month included',
 'settlement duration 37 72 month included',
 'settlement duration 73 108 month included',
 'settlement duration 109 month']

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'bow_model.joblib'
version_tfidf = 'tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)

tm.build_bow_model(preprocessing=True)
tm.build_tfidf_model(preprocessing=True)

tm.save_models('preprocessed_nums_')

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Bag-of-words model saved to preprocessed_nums_bow_model.joblib
TF-IDF model saved to preprocessed_nums_tfidf_model.joblib


In [6]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'A')]
targets = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'B')]
models = ['bow','tfidf','word2vec','sbert']
preprocessing = True
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'preprocessedbow_model.joblib'
version_tfidf = 'preprocessedtfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
n = 10

df = results_to_targets(descriptions=descriptions,targets=targets,model='bow',version=version_bow,
                        tm=tm, preprocessing=preprocessing,n=n,path=path, i=5)


dictsim = get_similarities_for_values(values=descriptions, docs=targets,encoding_method='bow',version=version_bow,
                                      tm=tm, n=n)

dictsim['R4']

['settlement duration 109 month',
 'settlement duration 6 month included',
 'settlement duration 7 12 month included',
 'settlement duration 13 36 month included',
 'settlement duration 37 72 month included',
 'settlement duration 73 108 month included',
 'month arrears 1 month ago',
 'application status previous month',
 'settlement duration',
 'payment prediction stray payer active settlement']

In [22]:
from helper import *


word2vec_version = "word2vec-google-news-300"
w2v = api.load(word2vec_version)
words = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '
words = words.split()

embeddings = [w2v[word] for word in words if word in w2v]

sentence_emb = sum(embeddings)/w2v.vector_size

wrong = sum(embeddings)/len(embeddings)

dif = sentence_emb - wrong

dif









array([ 1.31492633e-02, -4.20540804e-03, -1.29327383e-02, -3.89890969e-02,
        2.00114567e-02,  6.81758597e-02, -2.17630267e-02,  5.33513799e-02,
       -8.37658867e-02, -4.65147849e-03,  5.87817281e-02,  3.67194675e-02,
        1.28513575e-02, -1.43598272e-02,  9.20590535e-02, -2.45275721e-02,
       -2.35330313e-02, -1.20365415e-02,  1.70432646e-02,  1.51344892e-02,
        2.24123746e-02, -1.83520168e-02,  9.59865376e-02,  5.83819486e-03,
       -1.24128275e-02,  3.04218760e-04,  1.75759532e-02, -8.76721665e-02,
        1.45472679e-02, -1.59729756e-02,  1.36605613e-02,  8.50527287e-02,
        4.67060395e-02,  2.25175750e-02, -1.60711668e-02,  1.20551288e-02,
        3.39188986e-02,  2.09134184e-02, -2.33625975e-02, -2.07496490e-02,
       -3.72966938e-02,  7.42469123e-03, -5.57868853e-02, -2.32826415e-02,
        4.40336913e-02,  7.86332712e-02,  6.79416023e-03, -4.16686758e-02,
        1.65942125e-02, -1.28476759e-02, -1.46856140e-02, -2.88262637e-03,
        2.75255740e-02,  

In [20]:
L = [[2,1],[1,1],[1,1]]

array = np.array(L)

sum(array)/2


array([2. , 1.5])